In [1]:
import numpy as nmp
import pandas as pnd
import matplotlib.pyplot as plt

import pymc3 as pmc

import clonosGP as cln

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
DATA = pnd.read_csv('data/cll_Schuh_2012_CLL003.csv')

In [4]:
nmp.random.seed(42)
pmc.tt_rng(42)
RES = cln.infer(DATA, 
                model_args={'K': 20, 'prior': 'GP0', 'cov': 'Mat32', 'lik': 'BBin', 'threshold': 0.0}, 
                pymc3_args={'niters': 10000, 'method': 'advi', 'flow': 'scale-loc', 'learning_rate': 1e-2, 'random_seed': 42})

INFO:clonosGP:No PURITY column in the data. Assuming all samples have purity 100%.
INFO:clonosGP:No CNn column in the data. Assuming germline is diploid over all provided loci.
INFO:clonosGP:No CNt column in the data. Assuming all tumour samples are diploid over all provided loci.
INFO:clonosGP:No CNm column in the data. Multiplicity values will be approximated.
WARNING (theano.tensor.blas): We did not find a dynamic library in the library_dir of the library we use for blas. If you use ATLAS, make sure to compile it with dynamics library.
  0%|          | 0/10000 [00:00<?, ?it/s]/home/dimitris/VENVS/py37/lib/python3.7/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
Average Loss = 377.43: 100%|██

In [5]:
data = RES['data']
elbo = -RES['fit'].hist
ppd = RES['PPD']
weights = RES['weights']
centres = RES['centres']
centres_gp = RES['centres_gp']

weights

,CLUSTERID,W,W_LO,W_HI
0,1,3.580709e-01,1.957214e-01,0.566068
1,2,3.018461e-01,1.676488e-01,0.470401
2,3,2.916355e-01,1.610040e-01,0.465553
3,4,9.147366e-03,4.691054e-04,0.074482
4,5,2.781187e-03,4.573544e-05,0.038022
5,6,1.235638e-03,1.252427e-05,0.024856
6,7,5.185746e-04,3.150888e-06,0.013201
7,8,1.712914e-04,8.322174e-07,0.007735
8,9,7.008586e-05,2.590628e-07,0.005248
9,10,2.622278e-05,7.901845e-08,0.002383


In [6]:
%load_ext rpy2.ipython
%R library(tidyverse)
%R library(patchwork)


✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



array(['patchwork', 'forcats', 'stringr', 'dplyr', 'purrr', 'readr',
       'tidyr', 'tibble', 'ggplot2', 'tidyverse', 'tools', 'stats',
       'graphics', 'grDevices', 'utils', 'datasets', 'methods', 'base'],
      dtype='<U9')

In [8]:
%%R -i data,elbo,ppd,weights,centres,centres_gp -w 9 -h 9 --units in

gg1 =
    tibble(ITER=1:length(elbo), ELBO=elbo) %>%
    ggplot() +
    geom_line(aes(x = ITER, y = ELBO)) +
    labs(x='iteration', y='evidence lower bound') +
    theme_bw()
    

gg2 = 
    weights %>%
    ggplot() +
    geom_pointrange(aes(x = CLUSTERID, y = W, ymin = W_LO, ymax = W_HI)) +
    labs(x = 'cluster ID', y = 'cluster weights') +
    theme_bw()


cid = data %>% filter(CLUSTERID != 'uncertain') %>% pull(CLUSTERID)
centres = centres %>% mutate(CLUSTERID = as.factor(CLUSTERID)) %>% filter(CLUSTERID %in% cid)
centres_gp = centres_gp %>% mutate(CLUSTERID = as.factor(CLUSTERID)) %>% filter(CLUSTERID %in% cid)
gg3 =
    centres %>%
    ggplot() +
    geom_ribbon(aes(x = TIME, ymin = PHI_LO, ymax = PHI_HI, fill = CLUSTERID), data = centres_gp, alpha = 0.5, show_guide = FALSE) +
    geom_line(aes(x = TIME, y = PHI, color = CLUSTERID), data = centres_gp, show_guide = FALSE) +
    geom_point(aes(x = TIME2, y = PHI, color = CLUSTERID), show_guide = FALSE) +
    scale_x_continuous(breaks = unique(data$TIME2), labels = unique(data$SAMPLEID)) +
    scale_color_brewer(palette = 'Set2') +
    scale_fill_brewer(palette = 'Set2') +
    labs(x = 'sample', y = 'cancer cell fraction') +
    theme_bw()
 
gg4 =
    data %>%
    ggplot() +
    geom_line(aes(x = TIME2, y = VAF, group = MUTID, color = CLUSTERID)) +
    scale_x_continuous(breaks = unique(data$TIME2), labels = unique(data$SAMPLEID)) +
    scale_color_brewer(palette = 'Set2') +
    labs(x = 'sample', y = 'variant allele fraction', color = 'cluster ID') +
    theme_bw() +    
    theme(legend.background = element_blank(),
          legend.justification = c(0,1), legend.position = c(0,1), 
          legend.direction = 'horizontal') 

gg5 =
    data %>%
    ggplot() +
    geom_histogram(aes(x = VAF, y = stat(density)), breaks = seq(0, 1, by = 0.01)) +
    geom_line(aes(x = VAF, y = PPD), data = ppd, color = 'red') +
    facet_wrap(~SAMPLEID, nrow=1, labeller = labeller(SAMPLEID = function(x) str_c('Sample ', x))) +
    scale_x_continuous(limits = c(0, 1), labels = seq(0, 1, by=0.25)) +
#     scale_y_continuous(limits = c(0, 13)) +
    labs(x = 'variant allele fraction') +
    theme_bw()
    
((gg1 | gg2) / (gg3 + gg4) / gg5) +
    plot_annotation(tag_levels = 'A') +
    plot_layout(heights = c(1, 2, 0.5))
    
# ggsave('tmp.pdf')